In [1]:
using DifferentialEquations
using NLsolve
using Roots
using PyPlot
using Plots
using PyCall

# General Runge-Kutta for Second-Order ODEs

In [11]:
function RungeKutta(eqxns::Array{Function}, Stim::Int64, finish::Int64, dt::Float16) 
    """
    Give it an array of functions representing ODEs you want to numerically solve.
    Right now this only works for second-order ODEs represented with a 2x2 Jacobian
    """
    DT = dt # Time step
    Last = Int64(finish./DT + 1)
    Time = DT.*(0:Last-1) # Time vector
    WTS = [1 2 2 1]
    
    # Pre-initialize arrays for speed
    X = Array{Float64}(length(eqxns), length(Time))
    K = Array{Float64}(length(eqxns), 4)
    Weights = Array{Float64}(length(eqxns), 4)
    
    # THIS LOOP WORKS
    for NU = 1:2 # Two equations
        X[NU, :] = zeros(1, Last) 
        K[NU, :] = zeros(1, 4)
        Weights[NU, :] = WTS
    end
    
    Wt2 = [0 .5 .5 1] # Runge-Kutta weights
    rkIndex = [1 1 2 3] # Runge-Kutta index

    tic() # Start timing
    for T = 2:Last # For each time point
        for rk = 1:4 # Fourth order Runge-Kutta
            
            # Define X(i+1)
            XH = X[:, T-1] + K[:, rkIndex[rk]]*Wt2[rk]
            
            # Naka-Rushton: M = 100, N = 2, σ = 30
            j = 1
            for eq in eqxns
                K[j, rk] = DT*eq(XH[1], XH[2], Stim)[1]
                j += 1
            end

        end
        # Assign the time point X[:, T]
        X[:, T] = X[:, T-1] + sum((Weights.*K)', 1)'/6
    end
    toc()
    
    return X, Time
end

RungeKutta (generic function with 4 methods)

## 9.1 MM, GG and RinzelHH 
Based on equation 9.3

In [12]:
function MM(V)
    # Na action variable
    U = -V - 70
    Alpha = 0.1*(U + 25)./(exp((U+25)/10) - 1)
    Beta = 4*exp(U/18)
    return Alpha./(Alpha + Beta)
end

function GG(V)
    # Equilibrium value of recovery variable W for Hodgkin-Huxley Equations
    S = 1.2714
    U = -V - 70 # for new Rinzel system
    AH = 0.07*exp(U/20)
    BH = (1 + exp((U+30)/10)).^(-1)
    Hh = AH./(AH + BH);  # Na+ inactivation variable
    AN = 0.01*(U+10)./(exp((U+10)/10) - 1)
    BN = 0.125*exp(U/80)
    Nn = AN./(AN + BN)
    return S*(Nn + S*(1 - Hh))./(1 + S^2)
end



GG (generic function with 1 method)

# RinzelHH.m

In [14]:

#This particular variation has VNa = +55; VK = -92, and VL = -50.52
#Solution of Rinzel-Hodgkin-Huxley Equations
#Tempreature of 16.3 deg C

global IE VX
DT = 0.02
Last = 1000
Time = DT.*(1:Last) #Time vector

# Vectors to store results for V and W, use X[1, :] for V and X[2, :] for W
V = zeros(1, Last) 
W = zeros(1, Last)
#0<Input<10 for spike generation
IE = 5#IE = input("Stimulating current (range 0-200): ");  #External Current

S = 1.2714
# Initial conditions for Voltage and recovery Variable
V[1] = -70;  #Put initial conditions here
W[1] = 0.40388;  #This is the resting value
TT1 = clock()
for T = 2:Last

    
    for RK = 1:2  # Second Order Runge-Kutta
        
        M = MM(V[T-1])
        Geq = GG[V[T-1]]
        
        M = MM[VV];  # This and following lines calculate V dependent terms
        Geq = GG[VV];  # infinity or equilibrium value
        Tau = 5*exp(-(VV+60)^2/55^2) + 1
        V[T] = V[T-1] + (RK/2)*DT*(IE - 120*M^3*(1-WW)*(VV-55) - 36*(WW/S)^4*(VV+92) - 0.3*(VV+50.528))
        W[T] = W[T-1] + (RK/2)*DT*(3*(Geq - WW)/Tau)
        VV = V[T]
        WW = W[T]
  end
end

# Scaled to approximate units
WS = (W - 0.40388)*100 - 70

figure(1) 
Pz = PyPlot.plot(Time, V, "-r'); set(Pz, 'LineWidth", 2)
axis([0, DT*Last, -100, 55])
xlabel("Time (ms)'); ylabel('V[t]'); title('Rinzel Approximation to Hodgkin-Huxley")
Wnull = (-99:4:61)

S = 1.2714
U = -Wnull - 70;  #for new Rinzel system()
AH = 0.07*exp(U/20)
BH = (1 + exp((U+30)/10)).^(-1)
Hh = AH./(AH + BH);  #Na+ inactiWnullation Wnullariable
AN = 0.01*(U+10)./(exp((U+10)/10) - 1)
BN = 0.125*exp(U/80)
Nn = AN./(AN + BN)
DW = S*(Nn + S*(1 - Hh))./(1 + S^2)

WW = zeros(1, 23)
Vplot = (-90:6:54)
for j = 1:25
  VX = Vplot[j()]
  WW[j()] = fmin("Vnull", 0.1, 0.95, 0)
end
figure(2), PF = plot(Wnull, DW, "-b', Vplot, WW, '-k', V, W, '-r'); set(PF, 'LineWidth", 2)
axis([-100, 60, 0, 1]); axis square
xlabel("V'); ylabel('R'); title('Phase Plane, dV/dt = 0 (black), dR/dt = 0 (blue) & limit cycle (red)")


0.9860308458558551